In [1]:
%pylab notebook
from oiutils import oifits, oimodels, tellcorr, dpfit
import os
from astropy.io import fits
import ipywidgets as widgets

Populating the interactive namespace from numpy and matplotlib


In [2]:
# HD 148937
directory = '/Users/amerand/DATA/Science/Sana/SV'
files = ['GRAVI.2016-06-18T04:50:57.798_singlescivis_singlesciviscalibrated.fits', 
         'GRAVI.2016-06-18T05:02:00.835_singlescivis_singlesciviscalibrated.fits',
         'GRAVI.2016-06-20T00:09:05.552_singlescivis_singlesciviscalibrated.fits', 
         'GRAVI.2016-06-20T00:20:08.589_singlescivis_singlesciviscalibrated.fits',
         'GRAVI.2016-09-15T23:58:28.961_singlescivis_singlesciviscalibrated.fits', 
         'GRAVI.2016-09-16T00:04:31.982_singlescivis_singlesciviscalibrated.fits'
        ]
files = [os.path.join(directory, f) for f in files]

# Telluric correction
this takes quite a while! 

No need to run every time, since the results are stored in the FITS files, in the "TELLURICS" binary table

In [3]:
ntabs = len(files)
outs = [widgets.Output() for i in range(len(files))]
tab = widgets.Tab(children=outs)
display(tab)

# -- wavelength ranges where we can have potentially astropysical lines
# -- will be excluded from the 
avoid = [(2.056, 2.060), (2.160, 2.173), (2.293, 2.300), (2.322, 2.328), (2.352, 2.357), (2.382, 2.389), (2.415, 2.420)]

for i,f in enumerate(files):
    with outs[i]:
        tab.selected_index = i
        tab.children[i].clear_output()
        tab.set_title(i, 'file %d'%i)
        tellcorr.gravity(f, quiet=False, save=True, avoid=avoid, wlmin=1.9, wlmax=2.5, fig=i+1)

# Load Data (first epoch)

In [6]:
data = [oifits.loadOI(f, insname='GRAVITY_SC') for f in files if '2016-06' in f]

**** /Users/amerand/DATA/Science/Sana/SV/GRAVI.2016-06-18T04:50:57.798_singlescivis_singlesciviscalibrated.fits insname= GRAVITY_SC **********
WAVELENGTH: 1742
OI_VIS {'K0G1', 'J2A0', 'K0A0', 'G1A0', 'J2G1', 'K0J2'} (1, 1742)
OI_VIS2 {'K0G1', 'J2A0', 'K0A0', 'G1A0', 'J2G1', 'K0J2'} (1, 1742)
OI_FLUX {'J2', 'A0', 'K0', 'G1'} (1, 1742)
TELLURICS (1742,)
OI_T3 {'K0G1A0', 'K0J2A0', 'J2G1A0', 'K0J2G1'} (1, 1742)
**** /Users/amerand/DATA/Science/Sana/SV/GRAVI.2016-06-18T05:02:00.835_singlescivis_singlesciviscalibrated.fits insname= GRAVITY_SC **********
WAVELENGTH: 1742
OI_VIS {'K0G1', 'J2A0', 'K0A0', 'G1A0', 'J2G1', 'K0J2'} (1, 1742)
OI_VIS2 {'K0G1', 'J2A0', 'K0A0', 'G1A0', 'J2G1', 'K0J2'} (1, 1742)
OI_FLUX {'J2', 'A0', 'K0', 'G1'} (1, 1742)
TELLURICS (1742,)
OI_T3 {'K0G1A0', 'K0J2A0', 'J2G1A0', 'K0J2G1'} (1, 1742)
**** /Users/amerand/DATA/Science/Sana/SV/GRAVI.2016-06-20T00:09:05.552_singlescivis_singlesciviscalibrated.fits insname= GRAVITY_SC **********
WAVELENGTH: 1742
OI_VIS {'K0G1', 'J

# Fit Data

In [5]:
# -- context dictionnary for the fit:
fit = {# -- observable to fit
       'obs':['|V|', 'DPHI', 'NFLUX', 'T3PHI'], 
       # -- where to fit list of tuples (lambda_min, lambda_max)
       'wl ranges': [(2.167-0.018, 2.167+0.018)],
       # -- minimum error: override the errors in data file
       'min error': {'NFLUX':0.05, '|V|':0.04, 'DPHI':2.0, 'T3PHI':4.0},
       # -- maximum error: ignore data which are too noisy
       #'max error': {'NFLUX':0.5, '|V|':0.2, 'DPHI':5.0, 'T3PHI':10.0, 'V2':0.05},
       # -- spectral resolution, in pixels (default is no convolution)
       'spec res pix': 1.4
}

for d in data:
    d['fit'] = fit
    
# -- first guess for the model
param = {'1,ud':0.2, '1,x':0.0, '1,y':0.0, '1,f':1.0, '1,line_1_wl0':2.166, '1,line_1_lorentzian':1, '1,line_1_f':.5,
         '2,ud':0.2, '2,x':3.3, '2,y':1.5, '2,f':0.5, #'2,line_1_wl0':2.167, '2,line_1_gaussian':1, '2,line_1_f':.5,
        }
# -- parameters which will not be fitted
doNotFit = ['1,ud', '2,ud', # stars "1" and "2" are unresolved
            '1,f', # -- flux referenced to star "1"
            '1,x', '1,y', # -- star "1" is at the center of the field
           ]

outsF = [widgets.Output() for i in range(len(data)+1)]
tabF = widgets.Tab(children=outsF)
display(tabF)

with outsF[0]:
    tabF.selected_index = 0
    tabF.children[0].clear_output()
    tabF.set_title(0, 'Fitting')
    # -- zero vector to fit residuals
    z = oimodels.residualsOI(data, param)*0.0 
    # -- actual fit to data
    fit = dpfit.leastsqFit(oimodels.residualsOI, data, param, z, verbose=2, maxfev=2000, doNotFit=doNotFit, ftol=1e-4,)
    # -- best fit parameters
    param = fit['best']

# -- show data and image of the model 
for i in range(len(data)):
    with outsF[i+1]:
        tabF.selected_index = i+1
        tabF.children[i+1].clear_output()
        tabF.set_title(i+1, 'data %d'%i +(' + model' if i==len(data)-1 else ''))
        oimodels.showOI(data[i], param, fig=i+1, showIm=i==len(data)-1, plotWidth=12,  fov=8.0, pix=0.05, imPow=.1)
        
# plt.figure(1)
# plt.savefig('/Users/amerand/Desktop/Plots/SANA/HD148937_data.pdf')
# plt.figure(5)
# plt.savefig('/Users/amerand/Desktop/Plots/SANA/HD148937_model.pdf')
